In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np

print(tf. __version__)

In [ ]:
"""
一般結構：
（1）建立計算圖

（2）執行計算圖
"""




In [ ]:
#dataset
#(1)
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
X_train = mnist.train.images
Y_train = mnist.train.labels
X_test = mnist.test.images
Y_test = mnist.test.labels



In [ ]:
#model

def weight(shape):
    return tf.Variable(tf.truncated_normal(shape , stddev=0.1),name = 'W')
def bias(shape):
    return tf.Variable(tf.constant(0.1,shape=shape),name='b')
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#tf.name_scope:给计算图中的一组操作命名
with tf.name_scope('Input_Layer'):
    x=tf.placeholder("float",[None , 784],name='x')
    x_image = tf.reshape(x,[-1,28,28,1])

with tf.name_scope('C1_Conv'):
    W1=weight([5,5,1,16])
    b1=bias([16])
    Conv1=conv2d(x_image,W1)+b1
    C1_Conv = tf.nn.relu(Conv1)
with tf.name_scope('C1_pool'):
    C1_pool=max_pool_2x2(C1_Conv)

with tf.name_scope('C2_Conv'):
    W2=weight([5,5,16,36])
    b2=bias([36])
    Conv2=conv2d(C1_pool,W2)+b2
    C2_Conv = tf.nn.relu(Conv2)
with tf.name_scope('C2_pool'):
    C2_pool=max_pool_2x2(C2_Conv)

with tf.name_scope('D_Flat'):
    D_Flat = tf.reshape(C2_pool,[-1,1764])

with tf.name_scope('hidden'):
    W3=weight([1764,128])
    b3=bias([128])
    hidden=tf.nn.relu(tf.matmul(D_Flat,W3)+b3)
    dropout=tf.nn.dropout(hidden,keep_prob=0.8)

with tf.name_scope('OUTPUT'):
    W4=weight([128,10])
    b4=bias([10])
    y=tf.nn.softmax(tf.matmul(dropout,W4)+b4)


In [ ]:
#train

with tf.name_scope('optimizer'):
    #None表示在训练过程中可以使用任意数量的样本，而10则表示标签的维度，即表示有10个不同的分类标签。
    #placeholder:
    y_label=tf.placeholder("float",[None,10], name='y_label')
    #计算整个训练集的平均损失
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y,labels=y_label))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
    
with tf.name_scope('eva'):
    #比较模型的预测结果和真实标签
    #因为y_label是一个one-hot编码的向量，因此在第二个参数中指定axis=1，表示在每个样本的第一维度（即列）中查找最大值的索引。
    correct_prediction = tf.equal(tf.argmax(y_label , 1), tf.argmax(y,1))
    #计算模型在当前批次上的准确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction , "float"))

train = 10 #訓練周期數
batchSize= 100
total= 550 #一輪共幾個batch
loss_list=[];epoch_list=[];accuracy_list=[]

sess=tf.Session() #建立連結
#初始化所有全局变量
sess.run(tf.global_variables_initializer()) #初始化所有全局变量
#開始訓練
for epoch in range(train):
    print("In epoch:",epoch)
    #each batch
    for i in range(total):
        print("batch:",i+1)
        batch_x,batch_y=mnist.train.next_batch(batchSize) #从MNIST数据集的训练集中获取一个大小为batchSize的批次数据
        #optimizer是指optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)這個
        #执行前向传播和反向传播，并更新网络的参数
        sess.run(optimizer,feed_dict={x:batch_x , y_label:batch_y})
        
    #使用驗證資料計算準確機率以及誤差。
    #执行loss和accuracy操作
    loss1,acc=sess.run([loss,accuracy], feed_dict={x:mnist.validation.images, y_label:mnist.validation.labels})
    
    epoch_list.append(epoch)
    loss_list.append(loss1)
    accuracy_list.append(acc)
    print("Train Epoch:",'%02d' % (epoch+1),"loss=","{:.9f}".format(loss1)," accuracy=",acc)
    
